# Introduction to Numba
Now that we know how to profile functions, we're ready to improve their performance!

## What is Numba?
* A Just-in-time (JIT) compiler for user defined, Python functions.
* Compiles python code to machine (binary) code, using the LLVM compiler.


## How to use `numba` to improve performance.
1. Identify the bottleneck function.
2. Use the `jit` decorator to compile the function in machine code. 
3. That's it!


## A naive example
* In general, numba can optimize code when written in its most basic form.
* We often need to write unrolled loops!

In [ ]:
# calculate the average of an array, unrolled
def average_unrolled(arr):
    total = 0
    for elem in arr:
        total += elem
    total /= len(arr)
    return total

In [ ]:
import numpy as np
arr = np.random.uniform(size=1000000)
print(average_unrolled(arr))

In [ ]:
unrolled_time = %timeit -o average_unrolled(arr)

Okay, let's start with numba!

In [ ]:
from numba import jit

In [ ]:
# The simplest way to jit a function
@jit
def average_jitted(arr):
    total = 0.0
    for elem in arr:
        total += elem
    total /= len(arr)
    return total


In [ ]:
# There's also an alternative, less common syntax. We will use it extensively in
# this notebook because it's more compact.
average_jitted = jit()(average_unrolled)

In [ ]:
average_jitted(arr)

In [ ]:
jitted_time = %timeit -o average_jitted(arr)

In [ ]:
unrolled_time.average / jitted_time.average

The jitted version should be way faster than the unrolled version. 

What about numpy?

In [ ]:
%timeit arr.mean()

Numpy at its core runs compiled, optimized and usually parallelized with MKL, code.

Numba is best suited for compute intensive calculations.

Let's see a more complex example from BLonD. This is the Energy `kick` function:

In [ ]:
import math
import numpy as np

# Unrolled version of the kick (simplified for 1 RF station)
def kick_unrolled(dt, dE, voltage, omega, phi, acc_kick):
    for i in range(len(dt)):
        dE[i] += voltage * math.sin(omega * dt[i] + phi) + acc_kick

# Numpy version of kick (simplified for 1 RF station)
def kick_numpy(dt, dE, voltage, omega, phi, acc_kick):    
    dE += voltage * np.sin(omega * dt + phi) + acc_kick

# Random initialization of the necessary arrays
dE = np.random.uniform(size=1000000)
dt = np.random.uniform(size=1000000)
voltage = float(np.random.uniform(size=1))
phi = float(np.random.uniform(size=1))
omega = float(np.random.uniform(size=1))
acc_kick = float(np.random.uniform(size=1))


In [ ]:
unrolled_kick_time = %timeit -o kick_unrolled(dt, dE, voltage, omega, phi, acc_kick)


In [ ]:
numpy_kick_time = %timeit -o kick_numpy(dt, dE, voltage, omega, phi, acc_kick)

In [ ]:
jitted_kick = jit()(kick_unrolled)
jitted_kick_time = %timeit -o jitted_kick(dt, dE, voltage, omega, phi, acc_kick)

In [ ]:
numpy_speedup = unrolled_kick_time.best / numpy_kick_time.best
jit_speedup = unrolled_kick_time.best / jitted_kick_time.best
print(f'Numpy speedup: {numpy_speedup:.2f}')
print(f'Jit Speedup: {jit_speedup:.2f}')

## Diving deeper on Numba
Let's understand a little better how Numba works.

In [ ]:
# A simple math operation
def add(a, b):
    return a + b

In [ ]:
# Jit the add funtion
add_jit = jit()(add)
# inspect_types: Report information about the compiled function and the inferred types.
add_jit.inspect_types()

Returns an empty string, but why? What else is missing?

In [ ]:
add_jit(1, 1)

In [ ]:
add_jit.inspect_types()

In [ ]:
add_jit(1., 1.)

In [ ]:
add_jit.inspect_types()

Inspect types will list all input types that our jitted function has been compiled for!

In [ ]:
# Will this work?
print(add_jit('ab', 'c'))
add_jit.inspect_types()


In [ ]:
# Let's define a custom type (class) with an add method

class Point:
    def __init__(self, x = 0, y = 0):
        self.x = x
        self.y = y
    
    # For nice printing
    def __str__(self):
        return f'{self.x},{self.y}'
    
    # Overload + operator for Point
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        return Point(x, y)


In [ ]:
# we add points together in pure python
print(Point(1, 2) + Point(3, 4))

In [ ]:
# But can we add points in jitted functions?
res = add_jit(Point(1,2), Point(3, 4))
print(res)

In [ ]:
add_jit.inspect_types()

Numba has two compilation modes, the nopython mode and the pyobject mode. 
To enable nopython simply pass the argument: `nopython=True` to the jit decorator.

Nopython is a subset of python that numba knows how to deal with. Pyobjects ensure that everything will run, but very slowly in some cases. 

In [ ]:
add_njit = jit(nopython=True)(add)

In [ ]:
res = add_njit(Point(1,2), Point(3,4))
print(res)

By default jit will attempt to infer types, if it fails it will not compile the 

function, but run in python mode. This will not crash, produce correct results, 

but will not give any performance improvement. 

With `nopython=True` it will convert this warning to error, and will fail if 

numba cannot infer the variable type. This is often the required behavior, 

hence `numba.njit` is a synonym for jit(nopython=True)

## Global variables and Numba

In [ ]:
from numba import njit
# assume we have a global variable

secure_pass = '1234'

def get_access(input_pass):
    if secure_pass == input_pass:
        return 'Access Granted!'
    else:
        return 'Access Denied!'

get_access_jit = njit()(get_access)


In [ ]:
print(get_access('1234'))

In [ ]:
print(get_access_jit('1234'))

In [ ]:
# It is time to update the secure password
secure_pass = '5678'

In [ ]:
print(get_access('5678'))
print(get_access_jit('5678'))

Hmm, what has happened here?

So Numba treats globals as compile-time constants. In Python globals are not constants, which can lead to problems. 
The solution is to not use globals. Or:

In [ ]:
get_access_jit.recompile()
print(get_access_jit('5678'))

## `ufuncs` and `vectorize`
`ufuncs` or universal functions are functions that operate on ndarrays in an elementy-by-element fashion. 

Numpy uses `ufuncs` to allow for all sorts of array operations. 

Assume we want to calculate the standard normal propability distribution function for a given input. 

$$ PDF(x) = {1 \over \sigma \sqrt{2\pi} } e^{-{1 \over 2}({x-\mu \over \sigma})^2 }, \text{with } \mu = 0 \text{ and } \sigma = 1$$


In [ ]:
# This function will calculate the PDF of an input element x
import math
def standard_normal_pdf(x):
    return math.exp(-x**2/2.) / math.sqrt(2*math.pi)

In [ ]:
standard_normal_pdf(0.)


In [ ]:
# Will this run?
import numpy as np
x_arr = np.linspace(-4, 4, num=1000000)

standard_normal_pdf(x_arr)


To make it possible to operate on arrays, we just need to annotate the function with the `vectorize` decorator.

In [ ]:
from numba import vectorize

@vectorize
def vec_pdf(x):
    return math.exp(-x**2/2.) / math.sqrt(2*math.pi)

# or alternatively, with the in-line syntax
vec_pdf = vectorize()(standard_normal_pdf)


In [ ]:
# What if we try with a single element?
vec_pdf(0.)


In [ ]:
# Or with a vector?
vec_pdf(x_arr)


In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
# for x in x_arr[::50]:
plt.plot(x_arr, vec_pdf(x_arr), color='tab:blue')


In [ ]:
# Normal PDF with numpy
import numpy as np

def numpy_pdf(x):
    return np.exp(-x**2/2.) / np.sqrt(2*math.pi)


In [ ]:
# Test if the numpy and the vectorized numba implementations agree

print(np.allclose(vec_pdf(x_arr), numpy_pdf(x_arr)))


In [ ]:
# Normal PDF unrolled loop

def unrolled_pdf(x_arr):
    ret_arr = np.empty_like(x_arr)
    for i in range(len(x_arr)):
        ret_arr[i] = vec_pdf(x_arr[i])
    return ret_arr

jit_unrolled_pdf = njit()(unrolled_pdf)


In [ ]:
# No Numba, should be really slow, never try this
unrolled_time = %timeit -o unrolled_pdf(x_arr)

In [ ]:
# With Numba
jit_time = %timeit -o jit_unrolled_pdf(x_arr)

In [ ]:
# With Numpy
numpy_time = %timeit -o numpy_pdf(x_arr)

In [ ]:
# With Numba, with vectorize
vec_time = %timeit -o vec_pdf(x_arr)

# Automatic Parallelization with Numba

With Numba, you can easily parallelize jitted functions.
 
There are two ways to do this:
1. Passing the `target=parallel` argument to the `vectorize` decorator.
2. Passing the `parallel=True` argument to the `jit` or `njit` decorators.  

In [ ]:
# On top of vectorize, we can also parallelize the compiled machine code!
parallel_vec_pdf = vectorize(['float32(float32)', 'float64(float64)'],
                             target='parallel')(standard_normal_pdf)
# Only with a signature list we can use the target='parallel' kwarg


In [ ]:
parallel_vec_time = %timeit -o parallel_vec_pdf(x_arr)


In [ ]:
from numba import njit
parallel_jit_pdf = njit(parallel=True)(unrolled_pdf)


In [ ]:
parallel_jit_time = %timeit -o parallel_jit_pdf(x_arr)


In fact automatic parallelization is not that simple. We need to make sure there are no interloop dependencies. We can do this by the special parallel `range` generator provided by numba, called `prange`. 

In [ ]:
from numba import prange
@njit(parallel=True)
def parallel_jit_pdf(x_arr):
    ret_arr = np.empty_like(x_arr)
    for i in prange(len(x_arr)):
        ret_arr[i] = vec_pdf(x_arr[i])
    return ret_arr


In [ ]:
parallel_jit_time = %timeit -o parallel_jit_pdf(x_arr)


In [ ]:
# let's report all the collected timings

from prettytable import PrettyTable

rows = []
rows.append(['unrolled', unrolled_time.average, 1])
rows.append(['jit', jit_time.average, rows[-1][1]/jit_time.average])
rows.append(['vec', vec_time.average, rows[-1][1]/vec_time.average])
rows.append(['numpy', numpy_time.average, rows[-1][1]/numpy_time.average])
rows.append(['parallel_jit', parallel_jit_time.average,
            rows[-1][1]/parallel_jit_time.average])
rows.append(['parallel_vec', parallel_vec_time.average,
            rows[-1][1]/parallel_vec_time.average])



table = PrettyTable(['Version', 'Avg. Time (sec)', 'Relative Improvement'])
table.align = 'l'
# table.border = False
for r in rows:
    r[1] = np.round(r[1], 4)
    r[2] = np.round(r[2], 3)

    table.add_row(r)
print(table)


With Numba you can easily run operations in parallel on your multicore machine!

Since it is so simple and efficient, we should always use it!

In [ ]:
y_arr = np.array([-1., 1.])

%timeit vec_pdf(y_arr)
%timeit parallel_vec_pdf(y_arr)
%timeit parallel_jit_pdf(y_arr)



Well..

## Summary
 
* Never unroll loops in normal Python, use Numpy instead. 
* Just by adding the @jit (or @njit) line we can get great performance gains!
* Both @vectorize and @jit are easy to parallelize, for further improvement in multicore machines. 

<br/><br/>
<br/><br/>
<br/><br/>
<br/><br/>







## Additional example, Polynomial equation solution

Let's try another example, calculating the solution of a second order polynomial equation:
$$ x_{1,2} = {-b \pm \sqrt{b^2 - 4ac} \over 2a} $$

In [ ]:
from numba import njit, prange, vectorize

def unrolled_discriminant(A, B, C):
    X = np.empty_like(A)
    for i in range(len(A)):
        X[i] = (-B[i] + math.sqrt(abs(B[i]**2 - 4 * A[i] * C[i])) ) / (2*A[i])
    return X

@njit(parallel=True)
def unrolled_explicitely_parallel_discriminant(A, B, C):
    X = np.empty_like(A)
    for i in prange(len(A)):
        X[i] = (-B[i] + math.sqrt(abs(B[i]**2 - 4 * A[i] * C[i]))) / (2*A[i])
    return X

def numpy_discriminant(A, B, C):
    return (-B + np.sqrt(np.abs(B**2 - 4 * A * C)))/(2 * A)

def discriminant(a, b, c):
    return (-b + math.sqrt(abs(b**2 - 4 * a * c))) / (2*a)


jit_discriminant = njit()(unrolled_discriminant)
vec_discriminant = vectorize()(discriminant)
parallel_vec_disciminant = vectorize('float64(float64, float64, float64)', target='parallel')(discriminant)
parallel_jit_discriminant = njit(parallel=True)(numpy_discriminant)


In [ ]:
a_arr = np.random.uniform(1, 2, 5000000)
b_arr = np.random.uniform(0, 1, 5000000)
c_arr = np.random.uniform(-2, -1, 5000000)


Okay, let's benchmark all the versions of our function to figure out which performs best!

In [ ]:
unrolled_time = %timeit -o unrolled_discriminant(a_arr, b_arr, c_arr)


In [ ]:
numpy_time = %timeit -o numpy_discriminant(a_arr, b_arr, c_arr)


In [ ]:
jit_time = %timeit -o jit_discriminant(a_arr, b_arr, c_arr)


In [ ]:
vec_time = %timeit -o vec_discriminant(a_arr, b_arr, c_arr)


In [ ]:
parallel_vec_time = %timeit -o parallel_vec_disciminant(a_arr, b_arr, c_arr)


In [ ]:
parallel_jit_time = %timeit -o parallel_jit_discriminant(a_arr, b_arr, c_arr)


In [ ]:
unrolled_explicitely_parallel_time = %timeit -o unrolled_explicitely_parallel_discriminant(a_arr, b_arr, c_arr)


In [ ]:
from prettytable import PrettyTable

rows = []
rows.append(['unrolled', unrolled_time.average, 1])
rows.append(['numpy', numpy_time.average, rows[-1][1]/numpy_time.average])
rows.append(['jit', jit_time.average, rows[-1][1]/jit_time.average])
rows.append(['vec', vec_time.average, rows[-1][1]/vec_time.average])
rows.append(['parallel_vec', parallel_vec_time.average, rows[-1][1]/parallel_vec_time.average])
rows.append(['parallel_jit', parallel_jit_time.average, rows[-1][1]/parallel_jit_time.average])
rows.append(['unrolled_explicitely_parallel', unrolled_explicitely_parallel_time.average,
            rows[-1][1]/unrolled_explicitely_parallel_time.average])

table = PrettyTable(['Version', 'Avg. Time (sec)', 'Relative Improvement'])
table.align = 'l'
# table.border = False
for r in rows:
    r[1] = np.round(r[1], 4)
    r[2] = np.round(r[2], 3)

    table.add_row(r)
print(table)
